In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
with open('requirements.txt', 'w') as f:
    f.write('ultralytics>=8.2.0\n')

In [3]:
%pip install --upgrade pip wheel
%pip install -r requirements.txt
%pip install "ultralytics>=8.2.0" "opencv-python-headless>=4.8.0"
# Install torch with CUDA support
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu121

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [ultralytics]
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121


In [4]:
import zipfile
import os

# Specify the path to your zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/merged_dataset.yolov11.zip' # Corrected path

# Specify the directory where you want to extract the contents
# This should match the directory expected by your training script
extract_dir = '/content/merged_dataset.yolov11'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Unzip the file
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Dataset unzipped to {extract_dir}")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path}. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred during unzipping: {e}")

Dataset unzipped to /content/merged_dataset.yolov11


In [5]:
#!/usr/bin/env python3
"""Train a YOLOv11 model on the merged dataset.

This wrapper builds on the Ultralytics `YOLO` interface and provides a single entry-point
for launching training against the `merged_dataset.yolov11/data.yaml` file created by
`merge_yolov11_datasets.py`.
"""

import argparse
import ssl
import sys
import urllib.error
import urllib.request
from pathlib import Path

try:
    from ultralytics import YOLO
except ImportError as exc:  # pragma: no cover
    raise SystemExit(
        "Ultralytics is required. Install it with `pip install ultralytics`."
    ) from exc

try:
    import torch
except ImportError as exc:  # pragma: no cover
    raise SystemExit(
        "PyTorch is required. Install it with `pip install torch`."
    ) from exc


def parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="Train YOLOv11 on the merged dataset.")
    parser.add_argument(
        "--data",
        type=str,
        default="/content/merged_dataset.yolov11/merged_dataset.yolov11/data.yaml", # Corrected path
        help="Path to the dataset YAML file.",
    )
    parser.add_argument(
        "--model",
        type=str,
        default="yolo11n.pt",
        help="Name or path of the YOLOv11 checkpoint to fine-tune (e.g. yolo11n.pt, yolo11s.pt).",
    )
    parser.add_argument("--epochs", type=int, default=50, help="Number of epochs to train.")
    parser.add_argument("--img", type=int, default=640, help="Image size for training.")
    parser.add_argument("--batch", type=int, default=16, help="Batch size.")
    parser.add_argument(
        "--device",
        type=str,
        default="0", # Changed default to GPU
        help="Training device (e.g. '0' for GPU, 'cpu', 'mps', or 'auto').",
    )
    parser.add_argument(
        "--project",
        type=str,
        default="runs/train",
        help="Project directory for Ultralytics run outputs.",
    )
    parser.add_argument(
        "--name",
        type=str,
        default="yolov11-merged",
        help="Run name for Ultralytics outputs.",
    )
    parser.add_argument("--workers", type=int, default=8, help="Number of dataloader workers.")
    parser.add_argument("--patience", type=int, default=50, help="Early stopping patience.")
    parser.add_argument(
        "--save-period",
        type=int,
        default=-1,
        help="Save a checkpoint every N epochs (-1 disables periodic saves).",
    )
    parser.add_argument("--seed", type=int, default=0, help="Random seed.")
    parser.add_argument("--lr0", type=float, default=0.01, help="Initial learning rate.")
    parser.add_argument("--lrf", type=float, default=0.01, help="Final learning rate multiplier.")
    parser.add_argument(
        "--resume",
        action="store_true",
        help="Resume training from the last checkpoint in the run directory.",
    )
    parser.add_argument(
        "--exist-ok",
        action="store_true",
        help="Allow overwrite of existing project/name directory.",
    )
    # Add a known argument to consume the kernel-specific argument
    parser.add_argument("-f", "--fff", help="a dummy argument to absorb the kernel argument")
    return parser.parse_args()


def download_weights_if_needed(model_path: Path) -> Path:
    """Download the requested YOLO checkpoint from HuggingFace if missing."""

    alias_map = {
        "yolov11n.pt": "yolo11n.pt",
        "yolov11s.pt": "yolo11s.pt",
        "yolov11m.pt": "yolo11m.pt",
        "yolov11l.pt": "yolov11l.pt",
        "yolov11x.pt": "yolov11x.pt",
    }

    filename = model_path.name
    resolved_name = alias_map.get(filename, filename)
    target_path = model_path if resolved_name == filename else model_path.with_name(resolved_name)

    if target_path.exists():
        return target_path

    hf_url = f"https://huggingface.co/ultralytics/yolo11/resolve/main/{resolved_name}?download=1"
    print(f"Checkpoint '{resolved_name}' not found locally. Downloading from {hf_url} ...")

    context = ssl.create_default_context()
    try:
        cafile = __import__("certifi").where()  # type: ignore[assignment]
        context.load_verify_locations(cafile=cafile)
    except (ModuleNotFoundError, ssl.SSLError):
        pass

    try:
        with urllib.request.urlopen(hf_url, context=context) as response, target_path.open("wb") as dst:
            dst.write(response.read())
    except urllib.error.URLError as exc:
        raise SystemExit(
            f"Failed to download {resolved_name} from HuggingFace: {exc}. "
            "Download manually and re-run."
        ) from exc

    if target_path.stat().st_size < 1024:
        target_path.unlink(missing_ok=True)
        raise SystemExit(
            f"Downloaded file for {resolved_name} is unexpectedly small. "
            "Check the filename and try again."
        )

    print(f"Downloaded checkpoint to {target_path.resolve()}")
    return target_path


def main() -> int:
    args = parse_args()

    data_path = Path(args.data).expanduser().resolve()
    if not data_path.exists():
        raise SystemExit(f"Dataset YAML not found: {data_path}")

    model_path = Path(args.model)
    if not model_path.is_absolute():
        model_path = Path.cwd() / model_path
    model_path = download_weights_if_needed(model_path)

    model = YOLO(str(model_path))

    device_arg = args.device
    if device_arg == "auto" and not torch.cuda.is_available():
        print("No CUDA device detected. Using CPU.")
        device_arg = "cpu"
    elif device_arg != "cpu" and not torch.cuda.is_available():
         print(f"CUDA device {device_arg} not available. Using CPU instead.")
         device_arg = "cpu"


    train_kwargs = dict(
        data=str(data_path),
        epochs=args.epochs,
        imgsz=args.img,
        batch=args.batch,
        device=device_arg,
        project=args.project,
        name=args.name,
        workers=args.workers,
        patience=args.patience,
        save_period=args.save_period,
        seed=args.seed,
        lr0=args.lr0,
        lrf=args.lrf,
        exist_ok=args.exist_ok,
    )

    if args.resume:
        train_kwargs["resume"] = True

    print("Launching training with arguments:")
    for key, value in sorted(train_kwargs.items()):
        print(f"  {key}: {value}")

    results = model.train(**train_kwargs)
    print("Training finished. Metrics summary:")
    for metric, value in results.items():
        print(f"  {metric}: {value}")

    return 0

main() # Call main directly

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Checkpoint 'yolo11n.pt' not found locally. Downloading from https://huggingface.co/ultralytics/yolo11/resolve/main/yolo11n.pt?download=1 ...
Downloaded checkpoint to /content/yolo11n.pt
Launching training with arguments:
  batch: 16
  data: /content/merged_dataset.yolov11/merged_dataset.yolov11/data.yaml
  device: 0
  epochs: 50
  exist_ok: False
  imgsz: 640
  lr0: 0.01
  lrf: 0.01
  name: yolov11-merged
  patience: 50
  project: runs/train
  save_period: -1
  seed: 0
  workers: 8
Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cach

AttributeError: 'DetMetrics' object has no attribute 'items'. See valid attributes below.
Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        names (dict[int, str]): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (dict[str, float]): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
        stats (dict[str, list]): A dictionary containing lists for true positives, confidence scores, predicted classes,
            target classes, and target images.
        nt_per_class: Number of targets per class.
        nt_per_image: Number of targets per image.

    Methods:
        update_stats: Update statistics by appending new values to existing stat collections.
        process: Process predicted results for object detection and update metrics.
      

In [23]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
# Step 2: Unzip the second dataset

import zipfile
import os

# Specify the path to your second zip file in Google Drive
# Based on your previous zip, assuming it's in the main Drive folder
zip_file_path_2 = '/content/drive/MyDrive/merged_dataset_2.yolov11.zip'

# Specify the directory where you want to extract the contents of the second dataset
extract_dir_2 = '/content/merged_dataset_2.yolov11' # A new directory for the second dataset

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir_2, exist_ok=True)

print(f"Attempting to unzip {zip_file_path_2} to {extract_dir_2}")

# Unzip the file
try:
    with zipfile.ZipFile(zip_file_path_2, 'r') as zip_ref:
        zip_ref.extractall(extract_dir_2)
    print(f"Dataset unzipped successfully to {extract_dir_2}")
except FileNotFoundError:
    print(f"Error: Zip file not found at {zip_file_path_2}. Please check the path and try again.")
except Exception as e:
    print(f"An error occurred during unzipping: {e}")

Attempting to unzip /content/drive/MyDrive/merged_dataset_2.yolov11.zip to /content/merged_dataset_2.yolov11
Dataset unzipped successfully to /content/merged_dataset_2.yolov11


In [25]:
# Step 3: Verify the unzipped structure and update data.yaml paths

import yaml
import os

# Path to the data.yaml file in the second unzipped dataset
# Assuming the structure is similar to the first dataset
data_yaml_path_2 = os.path.join(extract_dir_2, 'merged_dataset_2.yolov11', 'data.yaml')

print(f"Verifying and updating data.yaml at: {data_yaml_path_2}")

try:
    # Load the data.yaml file
    with open(data_yaml_path_2, 'r') as f:
        data_yaml_content_2 = yaml.safe_load(f)

    # Get the base directory of the unzipped dataset
    dataset_base_dir_2 = os.path.dirname(data_yaml_path_2) # This is the directory containing data.yaml, train, valid, test

    # Update paths to be absolute
    for key in ['train', 'val', 'test']:
        if key in data_yaml_content_2:
            relative_path = data_yaml_content_2[key]
            # If the path is relative, make it absolute
            if isinstance(relative_path, str) and not os.path.isabs(relative_path):
                 # Check if the path is relative to the data.yaml's directory or the dataset root
                 # Assuming paths are relative to the directory containing data.yaml (e.g., 'train/images')
                absolute_path = os.path.join(dataset_base_dir_2, relative_path)
                data_yaml_content_2[key] = absolute_path
                print(f"Updated {key} path to: {absolute_path}")
            else:
                 print(f"{key} path is already absolute or not a string: {relative_path}")


    # Save the updated data.yaml file
    with open(data_yaml_path_2, 'w') as f:
        yaml.dump(data_yaml_content_2, f, default_flow_style=False)

    print("data.yaml updated with absolute paths.")
    print("\nUpdated data.yaml content:")
    print(yaml.dump(data_yaml_content_2, default_flow_style=False))

except FileNotFoundError:
    print(f"Error: data.yaml not found at {data_yaml_path_2}. Please check the unzipped structure.")
except Exception as e:
    print(f"An error occurred while processing data.yaml: {e}")

Verifying and updating data.yaml at: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/data.yaml
Updated train path to: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/train/images
Updated val path to: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/valid/images
Updated test path to: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/test/images
data.yaml updated with absolute paths.

Updated data.yaml content:
names:
- credit_card
- id_card
- passport
nc: 3
test: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/test/images
train: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/train/images
val: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/valid/images



In [26]:
# Step 4: Load the best trained model from the previous run

from ultralytics import YOLO

# Path to the best model from the previous training run
best_model_path = '/content/runs/train/yolov11-merged/weights/best.pt'

print(f"Loading model from: {best_model_path}")

try:
    model = YOLO(best_model_path)
    print("Model loaded successfully.")
except FileNotFoundError:
    print(f"Error: Best model not found at {best_model_path}. Please ensure the previous training completed and the file exists.")
except Exception as e:
    print(f"An error occurred while loading the model: {e}")

Loading model from: /content/runs/train/yolov11-merged/weights/best.pt
Model loaded successfully.


In [28]:
# Step 5: Define and run the fine-tuning training

# This code is adapted from the previous training script,
# but uses the loaded model and the second dataset's data.yaml

import argparse
import torch
import sys # Import sys to access sys.argv

# Define a function to parse arguments, similar to the original script
# We will provide the arguments programmatically
def parse_args_finetune(args_list):
    parser = argparse.ArgumentParser(description="Fine-tune YOLOv11 on the second merged dataset.")
    parser.add_argument(
        "--data",
        type=str,
        default="/content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/data.yaml", # Use the updated data.yaml path
        help="Path to the dataset YAML file for fine-tuning.",
    )
    parser.add_argument(
        "--model",
        type=str,
        default=None, # We load the model separately, so no default checkpoint file here
        help="Name or path of the YOLOv11 checkpoint to fine-tune (will be set from loaded model).",
    )
    parser.add_argument("--epochs", type=int, default=50, help="Number of epochs to fine-tune.") # Adjust epochs for fine-tuning
    parser.add_argument("--img", type=int, default=640, help="Image size for training.")
    parser.add_argument("--batch", type=int, default=16, help="Batch size.")
    parser.add_argument(
        "--device",
        type=str,
        default="0", # Use GPU by default for fine-tuning
        help="Training device (e.g. '0' for GPU, 'cpu', 'mps', or 'auto').",
    )
    parser.add_argument(
        "--project",
        type=str,
        default="runs/train_finetuned", # New project directory for fine-tuning results
        help="Project directory for Ultralytics run outputs.",
    )
    parser.add_argument(
        "--name",
        type=str,
        default="yolov11-merged-finetuned", # New run name
        help="Run name for Ultralytics outputs.",
    )
    parser.add_argument("--workers", type=int, default=8, help="Number of dataloader workers.")
    parser.add_argument("--patience", type=int, default=50, help="Early stopping patience.")
    parser.add_argument(
        "--save-period",
        type=int,
        default=-1,
        help="Save a checkpoint every N epochs (-1 disables periodic saves).",
    )
    parser.add_argument("--seed", type=int, default=0, help="Random seed.")
    parser.add_argument("--lr0", type=float, default=0.001, help="Initial learning rate for fine-tuning (often lower).") # Lower learning rate for fine-tuning
    parser.add_argument("--lrf", type=float, default=0.0001, help="Final learning rate multiplier for fine-tuning.") # Lower final learning rate
    parser.add_argument(
        "--resume",
        action="store_true",
        help="Resume training from the last checkpoint in the run directory.",
    )
    parser.add_argument(
        "--exist-ok",
        action="store_true",
        default=True, # Allow overwriting the fine-tuning run directory for simplicity
        help="Allow overwrite of existing project/name directory.",
    )
    # Parse known arguments and ignore the rest
    args, unknown = parser.parse_known_args(args_list)
    return args


# Define the arguments to pass to parse_args_finetune
# We are not passing arguments from the command line, but setting them here
# The actual arguments will come from sys.argv when run in Colab
finetune_args = [
    '--data', '/content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/data.yaml',
    '--epochs', '50', # Example: fine-tune for 50 epochs
    '--batch', '16',
    '--img', '640',
    '--device', '0',
    '--project', 'runs/train_finetuned',
    '--name', 'yolov11-merged-finetuned',
    '--workers', '8',
    '--patience', '50',
    '--save-period', '-1',
    '--seed', '0',
    '--lr0', '0.001', # Lower learning rate for fine-tuning
    '--lrf', '0.0001', # Lower final learning rate
    '--exist-ok', # Allow overwriting
    # Remove the '-f' argument if it exists in sys.argv before parsing
]

# Parse the arguments programmatically
args = parse_args_finetune(sys.argv[1:])


# Ensure a CUDA device is available if device is set to GPU
device_arg = args.device
if device_arg != "cpu" and not torch.cuda.is_available():
     print(f"CUDA device {device_arg} not available. Using CPU instead.")
     device_arg = "cpu"
elif device_arg == "auto" and not torch.cuda.is_available():
    print("No CUDA device detected. Using CPU.")
    device_arg = "cpu"


train_kwargs_finetune = dict(
    data=args.data,
    epochs=args.epochs,
    imgsz=args.img,
    batch=args.batch,
    device=device_arg,
    project=args.project,
    name=args.name,
    workers=args.workers,
    patience=args.patience,
    save_period=args.save_period,
    seed=args.seed,
    lr0=args.lr0,
    lrf=args.lrf,
    exist_ok=args.exist_ok,
)

if args.resume:
    train_kwargs_finetune["resume"] = True

print("Launching fine-tuning with arguments:")
for key, value in sorted(train_kwargs_finetune.items()):
    print(f"  {key}: {value}")

# Run the training (fine-tuning)
# The model object is already loaded in the previous cell
results_finetune = model.train(**train_kwargs_finetune)

print("Fine-tuning finished. Metrics summary:")
# Access and print metrics from the DetMetrics object
print(f"  Precision: {results_finetune.box.p}")
print(f"  Recall: {results_finetune.box.r}")
print(f"  mAP50: {results_finetune.box.map50}")
print(f"  mAP50-95: {results_finetune.box.map}")

Launching fine-tuning with arguments:
  batch: 16
  data: /content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/data.yaml
  device: 0
  epochs: 50
  exist_ok: True
  imgsz: 640
  lr0: 0.001
  lrf: 0.0001
  name: yolov11-merged-finetuned
  patience: 50
  project: runs/train_finetuned
  save_period: -1
  seed: 0
  workers: 8
Ultralytics 8.3.226 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset_2.yolov11/merged_dataset_2.yolov11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=